In [1]:
!pip install pymongo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.3/671.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 8.9 MB/s eta 0:00:00


In [ ]:
# row 생략 없이 출력
pd.set_option('display.max_rows', None)
# col 생략 없이 출력
pd.set_option('display.max_columns', None)

In [2]:
import pandas as pd
import numpy as np
from google.colab import drive
# 구글드라이브 마운트 경로 설정
drive_path = '/content/drive/MyDrive/data/'

In [3]:
# CSV 파일 로드
exchange = pd.read_table(drive_path + 'exchange.csv', sep=",", thousands=',')
crime = pd.read_table(drive_path + 'crime_rate_korean_encoded_final (1).csv', sep=",", thousands=',')
price = pd.read_table(drive_path + '국가별_물가지수_예측.csv', sep=",", thousands=',')
climate = pd.read_table(drive_path + 'climate_prediction.csv', sep=",", thousands=',', encoding='CP949')
traveler = pd.read_table(drive_path + 'traveler_trends_prediction.csv', sep=",", thousands=',', encoding='CP949')

In [4]:
import pymongo
import pandas as pd
import json
client = pymongo.MongoClient("mongodb://j9a305.p.ssafy.io/",
                             username = 'm1z5',
                             password='ssafy1357')
db = client["tripickDB"]
print(db)

Database(MongoClient(host=['j9a305.p.ssafy.io:27017'], document_class=dict, tz_aware=False, connect=True), 'tripickDB')


In [ ]:
climate.head()

,country,city,date,temp_avg,temp_max,temp_min,rain_max,rainy_days
0,중국,광저우,2023-09-01,31.36,30.36,28.78,62.88,10.28
1,중국,광저우,2023-10-01,29.47,28.47,26.94,70.67,12.77
2,중국,광저우,2023-11-01,24.44,23.38,21.62,41.19,13.00
3,중국,광저우,2023-12-01,19.53,18.15,17.05,42.90,8.81
4,중국,광저우,2024-01-01,19.33,19.11,16.59,53.67,8.01


In [ ]:
# cities 컬렉션
service_cities = []
traverler_list = traveler[traveler['date']=='2023-09-01']
traverler_list = traverler_list[['city','country']]
climate_list = climate[climate['date']=='2023-09-01']
climate_list = climate_list[['city','country']]

for _,row in traverler_list.iterrows():
  city_name = row['city']
  service_cities.append(city_name)

for _,row in climate_list.iterrows():
  city_name = row['city']
  if city_name not in service_cities:
    service_cities.append(city_name)

# 도시명, 국가명, 기후 리스트, 여행객 동향 리스트 필요
for city_name in service_cities:
    # 현재 국가, 도시명
    # country_name = row['country']
    # 데이터프레임 to 딕셔너리
    by_trip = traverler_list[traverler_list['city']==city_name]
    by_cliamate = climate_list[climate_list['city']==city_name]
    if by_trip.empty:
       country_name = by_cliamate['country'].iloc[0]
    else:
      country_name = by_trip['country'].iloc[0]
    traveler_tmp = traveler[traveler['city']==city_name]
    climate_tmp = climate[climate['city']==city_name]
    traveler_tmp = traveler_tmp[['date','people']]
    climate_tmp = climate_tmp[['date','temp_avg'	,'temp_max',	'temp_min'	,'rain_max',	'rainy_days']]
    traveler_dict = traveler_tmp.to_dict(orient="records")
    climate_dict = climate_tmp.to_dict(orient="records")
    cities = {
        'name' : city_name,
        'country' : country_name,
        'traveler' : traveler_dict,
        'climate' : climate_dict
    }
    db.cities.insert_one(cities)
    print(cities)


In [ ]:
crime

,country,crimeIndex,population,country_korean
0,Venezuela,83.76,28838499,베네수엘라
1,Papua New Guinea,80.79,10329931,파푸아뉴기니
2,South Africa,76.86,60414495,남아프리카
3,Afghanistan,76.31,42239854,아프가니스탄
4,Honduras,74.54,10593798,온두라스
...,...,...,...,...
128,Switzerland,21.62,8796669,스위스
129,Oman,20.34,4644384,오만
130,Taiwan,15.46,23923276,대만
131,United Arab Emirates,15.23,9516871,아랍에미리트


In [5]:
# countries 컬렉션
service_contries = []
traverler_list = traveler[traveler['date']=='2023-09-01']
traverler_list = traverler_list[['country']]
climate_list = climate[climate['date']=='2023-09-01']
climate_list = climate_list[['country']]

for _,row in traverler_list.iterrows():
  country_name = row['country']
  if country_name not in service_contries:
    service_contries.append(country_name)

for _,row in climate_list.iterrows():
  country_name = row['country']
  if country_name not in service_contries:
    service_contries.append(country_name)

for column_name in exchange.columns:
  if column_name == 'date':
      continue
  if column_name not in service_contries:
    service_contries.append(column_name)

# 환율, 범죄율, 물가지수 데이터 필요
for country_name in service_contries:
    # 환율
    exchange_dict = {}
    if country_name in exchange.columns:
      exchange_tmp = exchange[['date',country_name]]
      exchange_tmp.head()
      exchange_tmp.rename(columns={country_name: 'exchange_rate'}, inplace=True)
      exchange_dict = {}
      for index, row in exchange_tmp.iterrows():
        exchange_dict[row['date']] = row['exchange_rate']

    # 물가지수
    price_tmp = price[price['country']==country_name]
    price_tmp = price_tmp[['2023','2024','2025']]
    price_dict = price_tmp.to_dict(orient="records")
    # 범죄율
    crime_tmp = crime[crime['country_korean']==country_name]
    if not crime_tmp.empty:
      crime_rate = crime_tmp.iloc[0]
      crime_rate = crime_rate[['crimeIndex']].iloc[0]
    countries = {
        'name' : country_name,
        'crime_rate' : crime_rate,
        'price_index': price_dict,
        'exchange_rate' : exchange_dict
    }
    db.countries.insert_one(countries)
    print(countries)

{'name': '네덜란드', 'crime_rate': 27.16, 'price_index': [{'2023': 121.45475913858428, '2024': 123.15081923914111, '2025': 124.84685932660685}], 'exchange_rate': {}, '_id': ObjectId('65095785fa1b4f8bc4c9e92b')}
{'name': '뉴질랜드', 'crime_rate': 42.88, 'price_index': [{'2023': 121.00972139199048, '2024': 122.7648265774999, '2025': 124.51868037312744}], 'exchange_rate': {'2023-09-01': 790.7174985154074, '2023-09-02': 791.1184584336942, '2023-09-03': 790.5646428518404, '2023-09-04': 790.6140184036391, '2023-09-05': 790.8885779039678, '2023-09-06': 790.9695550806371, '2023-09-07': 791.6677743495815, '2023-09-08': 791.367290621771, '2023-09-09': 791.3799666572739, '2023-09-10': 791.3891719445421, '2023-09-11': 791.4338970602042, '2023-09-12': 791.7413929745461, '2023-09-13': 791.981725814783, '2023-09-14': 791.9734832112325, '2023-09-15': 792.0691839367892, '2023-09-16': 792.6239913003656, '2023-09-17': 792.3802654621013, '2023-09-18': 792.3163305508588, '2023-09-19': 792.863064983093, '2023-09-20

<ipython-input-5-185a0b1b5d01>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exchange_tmp.rename(columns={country_name: 'exchange_rate'}, inplace=True)


{'name': '대만', 'crime_rate': 15.46, 'price_index': [{'2023': 105.91045759913284, '2024': 106.9670805025666, '2025': 107.40200765439852}], 'exchange_rate': {}, '_id': ObjectId('65095785fa1b4f8bc4c9e92d')}
{'name': '독일', 'crime_rate': 35.79, 'price_index': [{'2023': 118.73527083196429, '2024': 120.25222011468767, '2025': 121.76898330529828}], 'exchange_rate': {}, '_id': ObjectId('65095785fa1b4f8bc4c9e92e')}
{'name': '라오스', 'crime_rate': 35.79, 'price_index': [{'2023': 142.84151557897772, '2024': 144.68086155079567, '2025': 146.4172411768149}], 'exchange_rate': {}, '_id': ObjectId('65095785fa1b4f8bc4c9e92f')}
{'name': '러시아', 'crime_rate': 39.99, 'price_index': [{'2023': 208.2135524809592, '2024': 215.00581053054253, '2025': 221.6643942179204}], 'exchange_rate': {}, '_id': ObjectId('65095786fa1b4f8bc4c9e930')}


<ipython-input-5-185a0b1b5d01>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exchange_tmp.rename(columns={country_name: 'exchange_rate'}, inplace=True)


{'name': '말레이시아', 'crime_rate': 57.29, 'price_index': [{'2023': 131.9174842620253, '2024': 134.1285777683815, '2025': 136.39040215696352}], 'exchange_rate': {'2023-09-01': 284.00727563289786, '2023-09-02': 284.0041198143574, '2023-09-03': 283.9873772108071, '2023-09-04': 284.0622366782627, '2023-09-05': 283.9175585294883, '2023-09-06': 283.9143395945771, '2023-09-07': 283.81483910148035, '2023-09-08': 283.92886481294846, '2023-09-09': 283.9261330575244, '2023-09-10': 283.85920061176495, '2023-09-11': 283.5820546307736, '2023-09-12': 283.65293825996497, '2023-09-13': 283.4480043349156, '2023-09-14': 283.57936232308134, '2023-09-15': 283.5772617313653, '2023-09-16': 283.3121559269133, '2023-09-17': 283.4904250167978, '2023-09-18': 283.35580765979574, '2023-09-19': 283.1589278344928, '2023-09-20': 283.2440137741941, '2023-09-21': 283.23690332054525, '2023-09-22': 283.1534362171358, '2023-09-23': 283.15252891930544, '2023-09-24': 283.14449144396025, '2023-09-25': 283.1527281303508, '2023-0

...

{'name': '필리핀 ', 'crime_rate': 39.96, 'price_index': [], 'exchange_rate': {'2023-09-01': 23.3428337580959, '2023-09-02': 23.362255732444275, '2023-09-03': 23.36405701449556, '2023-09-04': 23.354337290629672, '2023-09-05': 23.350679992799304, '2023-09-06': 23.36495075216814, '2023-09-07': 23.36196107980347, '2023-09-08': 23.345062198814727, '2023-09-09': 23.360371089665406, '2023-09-10': 23.360371089665406, '2023-09-11': 23.340851724638707, '2023-09-12': 23.35696460957223, '2023-09-13': 23.35694414606138, '2023-09-14': 23.33931819056958, '2023-09-15': 23.33717494599364, '2023-09-16': 23.352405683049984, '2023-09-17': 23.33439819165455, '2023-09-18': 23.32989930328471, '2023-09-19': 23.34878863023101, '2023-09-20': 23.33486641653119, '2023-09-21': 23.324906810435944, '2023-09-22': 23.31914395548645, '2023-09-23': 23.32165361063692, '2023-09-24': 23.31166076402653, '2023-09-25': 23.309863567633187, '2023-09-26': 23.318974133362484, '2023-09-27': 23.31463607668233, '2023-09-28': 23.3143024

<ipython-input-5-185a0b1b5d01>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exchange_tmp.rename(columns={country_name: 'exchange_rate'}, inplace=True)
<ipython-input-5-185a0b1b5d01>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exchange_tmp.rename(columns={country_name: 'exchange_rate'}, inplace=True)


{'name': '스웨덴', 'crime_rate': 48.0, 'price_index': [{'2023': 116.84135701871594, '2024': 118.2439065449705, '2025': 119.56698818549368}], 'exchange_rate': {'2023-09-01': 122.26761124879404, '2023-09-02': 122.33347656296702, '2023-09-03': 122.30330179542688, '2023-09-04': 122.26515862129332, '2023-09-05': 122.22428164890674, '2023-09-06': 122.3292167010341, '2023-09-07': 122.39771315970364, '2023-09-08': 122.42528860349064, '2023-09-09': 122.4249635823579, '2023-09-10': 122.3386643623471, '2023-09-11': 122.1966295798219, '2023-09-12': 122.21009041385965, '2023-09-13': 122.21174968407662, '2023-09-14': 122.201365068692, '2023-09-15': 122.25419833158296, '2023-09-16': 122.26998745058364, '2023-09-17': 122.26621854854548, '2023-09-18': 122.25848777931476, '2023-09-19': 122.28647002783543, '2023-09-20': 122.2755873848374, '2023-09-21': 122.26595622112934, '2023-09-22': 122.2788635189597, '2023-09-23': 122.29144707642789, '2023-09-24': 122.2704924413194, '2023-09-25': 122.29613248076711, '20

<ipython-input-5-185a0b1b5d01>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exchange_tmp.rename(columns={country_name: 'exchange_rate'}, inplace=True)
<ipython-input-5-185a0b1b5d01>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exchange_tmp.rename(columns={country_name: 'exchange_rate'}, inplace=True)


{'name': '남아공', 'crime_rate': 48.0, 'price_index': [], 'exchange_rate': {'2023-09-01': 71.02912093782295, '2023-09-02': 71.04503415281307, '2023-09-03': 71.03095603221568, '2023-09-04': 71.02911114633235, '2023-09-05': 71.02877520663104, '2023-09-06': 71.00675837090265, '2023-09-07': 70.99096494317655, '2023-09-08': 70.98897980651832, '2023-09-09': 70.98396550671555, '2023-09-10': 70.97689661406973, '2023-09-11': 70.96846071663384, '2023-09-12': 70.9688925284118, '2023-09-13': 70.99006919789616, '2023-09-14': 70.95117422748194, '2023-09-15': 70.95662195923738, '2023-09-16': 70.9847787639711, '2023-09-17': 71.0924750164366, '2023-09-18': 71.0924750164366, '2023-09-19': 71.02300108011652, '2023-09-20': 71.04489111956423, '2023-09-21': 71.04489111956423, '2023-09-22': 71.02320205222134, '2023-09-23': 71.05152052221287, '2023-09-24': 71.04869853010243, '2023-09-25': 71.0406132243825, '2023-09-26': 71.13022924297928, '2023-09-27': 71.18353446980375, '2023-09-28': 71.18682342443884, '2023-09

In [ ]:
exchange_tmp

,date,exchange_rate
0,2023-09-01,71.029121
1,2023-09-02,71.045034
2,2023-09-03,71.030956
3,2023-09-04,71.029111
4,2023-09-05,71.028775
...,...,...
208,2024-03-27,71.877019
209,2024-03-28,71.877019
210,2024-03-29,71.877019
211,2024-03-30,71.877019


In [ ]:
exchange = exchange.to_dict(orient="records")

In [ ]:
# mongoDB에 데이터 삽입
# 데이터베이스.컬렉션명.insert_many(딕셔너리)
db.exchange.insert_many(exchange)